In [ ]:
import sqlite3
from selenium.webdriver.common.by import By
import os
from automation import Automation



def retrieve_img_from_db(img_id, db_name, table_name):
    with sqlite3.connect(db_name) as conn:
        cursor = conn.cursor()
        cursor.execute(f"SELECT img FROM {table_name} WHERE id = ?", (img_id,))
        img = cursor.fetchone()[0]
        return img

import sqlite3

def sort_db_by_gr_name(db_name, table_name):
    with sqlite3.connect(db_name) as conn:
        cursor = conn.cursor()

        # Retrieve all data sorted by gr_name
        cursor.execute(f"SELECT * FROM {table_name} ORDER BY gr_name ASC")
        sorted_data = cursor.fetchall()

        # Get column count dynamically
        cursor.execute(f"PRAGMA table_info({table_name})")
        num_columns = len(cursor.fetchall())

        # Clear the table
        cursor.execute(f"DELETE FROM {table_name}")

        # Reinsert sorted data
        placeholders = ", ".join(["?"] * num_columns)  # Generate (?, ?, ?, ?)
        cursor.executemany(f"INSERT INTO {table_name} VALUES ({placeholders})", sorted_data)

        conn.commit()



# sort_db_by_gr_name("instance/music.db", "artists")


def get_images_with_driver():
    db = "instance/music.db"
    table = "artists"
    # get artists from db that the artist_img is None
    conn = sqlite3.connect(db)
    cursor = conn.cursor()
    cursor.execute(f"SELECT gr_name FROM {table} WHERE artist_img IS NULL")
    artists = cursor.fetchall()
    artists = [artist[0] for artist in artists]


    automation = Automation()
    driver = automation.start_chrome_driver()
    patreon_url = "https://www.patreon.com/c/partitourabouz/collections"
    driver.get(patreon_url)
    input("Press Enter to continue...")
    artists_cards = driver.find_elements(By.CSS_SELECTOR, ".sc-ieecCq.hlVSeb")
    images_dict = {}
    print(len(artists_cards))
    for aritst_card in artists_cards:
        artist_name_elem = aritst_card.find_element(By.CSS_SELECTOR, '[data-tag="box-collection-title-href"]')
        artist_name = artist_name_elem.text.lower()
        if artist_name:
            artist_img_elem = aritst_card.find_element(By.CSS_SELECTOR, '[data-tag="box-collection-thumbnail"]')
            # get background-img url from artist_img_elem
            artist_img_url = artist_img_elem.value_of_css_property("background-image")
            if artist_img_url:
                artist_img_url = artist_img_url.replace('url("', "").replace('")', "")
                images_dict[artist_name] = artist_img_url

    for artist_name, artist_img_url in images_dict.items():
        os.makedirs("images", exist_ok=True)
        try:
            driver.get(artist_img_url)

            artist_image = driver.find_element(By.CSS_SELECTOR, "img").screenshot_as_png
        except Exception as e:
            print(e)
            continue
        else:
            if isinstance(artist_image, bytes):
                
                with open (f"images/{artist_name}.png", "wb") as file:
                    file.write(artist_image)

                # store_img_in_db(f"images/{artist_name}.png", "instance/music.db", "artists", artist_name)
            else: 
                    print("No image found")
                    print(type(artist_image))
            
    driver.quit()
    return images_dict




a
backing tracks
αντωνης κατιναρης
ασικης γρηγορης
βαμβακαρης μαρκος
βαμβακαρης στελιος
βασιλειαδης βασιλης
βασιλης δημητριου
βασιλης καραπατακης
βασιλης τσιτσανης
βασω αλλαγιαννη
βελλας ιωαννης
βυζας δημητρης
γαβαλας γιαννης
γαβαλας πανος
γενιτσαρης μιχαλης
γερασιμος κλουβατος
γιαννης πολυκανδριωτης
γιοβαν τσαους
γιωργος ζηκας
γιωργος σαμπανης
δελιας ανεστης
δερβενιωτης θεοδωρος
δημητρης γκογκος μπαγιαντερας
δημητρης ευσταθιου
δημητρης παπαδημητριου
ζαμπετας γιωργος
ζαφειριου στελιος
ζερβας λευτερης
ηλιας ανδριοπουλος
θανασης πολυκανδριωτης
θανος μικρουτσικος
θεοδωρακης μικης
θοδωρης παπαδοπουλος
ιακωβιδης ζακ
ιωσηφ ριτσιαρδης
καζαντζιδης στελιος
καλδαρας αποστολος
καπλανης κωστας
καρανικολας νικος
κιντατος λευτερης
κορακακης βαγγελης
κουγιουμτζης σταυρος
κωστας καριπης
κωστας λαβιδας
κωστας μπεζος - α
λαυκας γιωργος
λεμονοπουλος χαρης
λοϊζος μανος
μαλαμας σωκρατης
μανισαλης γιωργος
μανωλης αγγελοπουλος
μανωλης χιωτης
μαρκοπουλος γιαννης
μητσακης γιωργος
μιμης χριστοπουλος
μινως ματσα

In [32]:
def store_img_in_db(img_path, db_name, table_name):

    with sqlite3.connect(db_name) as conn:
        cursor = conn.cursor()
        # check if file_name from img path == gr_name in the db
        gr_name = img_path.split("/")[-1].split(".")[0]
        # caplitlize each word in gr_name
        gr_name = " ".join([word.capitalize() for word in gr_name.split()])
        cursor.execute(f"SELECT gr_name FROM {table_name} WHERE gr_name = ?", (gr_name,))
        if not cursor.fetchone():
            print(f"{gr_name} not found in db")
            return
        
        
        with open(img_path, "rb") as file:
            img = file.read()
        cursor.execute(f"UPDATE {table_name} SET artist_img = ? WHERE gr_name = ?", (img, gr_name))
        print(f"Stored {gr_name} in db")
        conn.commit()

# iter through the images in images folder and store them in the db
for img in os.listdir("images"):
    store_img_in_db(f"images/{img}", "instance/music.db", "artists")


A not found in db
Backing Tracks not found in db
Stored Αντωνης Κατιναρης in db
Ασικης Γρηγορης not found in db
Stored Βαμβακαρης Μαρκος in db
Stored Βαμβακαρης Στελιος in db
Stored Βασιλειαδης Βασιλης in db
Βασιλης Δημητριου not found in db
Βασιλης Καραπατακης not found in db
Stored Βασιλης Τσιτσανης in db
Βασω Αλλαγιαννη not found in db
Stored Βελλας Ιωαννης in db
Stored Βυζας Δημητρης in db
Stored Γαβαλας Γιαννης in db
Stored Γαβαλας Πανος in db
Stored Γενιτσαρης Μιχαλης in db
Γερασιμος Κλουβατος not found in db
Γιαννης Πολυκανδριωτης not found in db
Stored Γιοβαν Τσαους in db
Γιωργος Ζηκας not found in db
Stored Γιωργος Σαμπανης in db
Stored Δελιας Ανεστης in db
Stored Δερβενιωτης Θεοδωρος in db
Stored Δημητρης Γκογκος Μπαγιαντερας in db
Δημητρης Ευσταθιου not found in db
Δημητρης Παπαδημητριου not found in db
Stored Ζαμπετας Γιωργος in db
Stored Ζαφειριου Στελιος in db
Stored Ζερβας Λευτερης in db
Ηλιας Ανδριοπουλος not found in db
Stored Θανασης Πολυκανδριωτης in db
Stored Θανος 